# Pandas (Parte 2)

En la última sesión vimos una introducción a Pandas. 
Las cosas que aprendimos a hacer fueron:

- Crear y leer una base de datos. 
- Realizar los primeros análisis descriptivos de una base de datos. 
- Seleccionar información (por columnas  y filas). 
- Tabular información 
- Filtrar información 

### Ordenando valores 

A veces queremos ordenar nuestro dataframe de acuerdo a criterios específicos

## Limpieza de datos

In [2]:
import pandas as pd

In [3]:
locacion_datos = "https://otorongo.club/2021/json/ingresos/"

cong = pd.read_json(locacion_datos)

In [17]:

## Ordenando de menor a mayor
cong.sort_values("total_ingreso", inplace = True)
#cong.head(4)

In [16]:
## Ordenando de mayor a menor
cong.sort_values("total_ingreso",ascending = False,  inplace = True)
#cong.head(3)

In [15]:
## Ordenando con más de un criterio. 
cong.sort_values(['partido', 'total_ingreso'],ascending = False,  inplace = True)
#cong.head(10)

### Cambiando de nombre a nuestras columnas
Muchas veces necesitamos cambiar de nombre a nuestras columnas. Podemos hacerlo directamente: 

In [45]:
cong2 = cong.copy(deep = True)

In [19]:
cong2.columns

Index(['nombre', 'dni', 'partido', 'total_ingreso', 'ingreso_publico',
       'ingreso_privado', 'renta_publico', 'renta_privado',
       'otro_ingreso_publico', 'otro_ingreso_privado'],
      dtype='object')

In [25]:
cong2.columns = ['nombre_congresista', "dni_cong"] + list(cong2.columns[2:]) ## Esta lista al final 

Otra forma de cambiar nombres es definir un diccionario donde se da el ```nombre_viejo: nombre_nuevo ```

In [29]:
nuevos_nombres = {'nombre_congresista': 'nombre', 'dni_cong' : 'dni'}

In [32]:
cong2.rename(columns = nuevos_nombres, inplace = True)

### Cambiando el valor de las variables
Una forma práctica de cambiar los valores de nuestras variables, es usar un diccionario: 

In [38]:
cong2['partido_nuevo'] = cong2['partido']

cong2.replace({'partido_nuevo': {'ALIANZA PARA EL PROGRESO': 'FUERZA POPULAR',
                                 'JUNTOS POR EL PERU': 'PARTIDO POLITICO NACIONAL PERU LIBRE'}}, inplace = True)

También podemos usar .loc para reemplazar valores:

In [43]:
cong2.loc[cong2['partido'] == 'ALIANZA PARA EL PROGRESO', 'partido'] = 'FUERZA POPULAR'

Tambien podemos usar el comando ``` isin``` :

In [48]:
cong2['nueva_coalicion'] = 0
cong2.loc[cong2["partido"].isin(['ALIANZA PARA EL PROGRESO',
                                 'FUERZA POPULAR', 'VICTORIA NACIONAL' ]), "nueva_coalicion"] = 1

### Cómo crear variables nuevas

En este caso estoy utilizando indistintamente "variable" de "columna"

In [63]:
cong['nueva columna'] = True
cong.head(3)

,nombre,dni,partido,total_ingreso,ingreso_publico,ingreso_privado,renta_publico,renta_privado,otro_ingreso_publico,otro_ingreso_privado,new_ingreso_mp,new_ingreso_mensual,nueva columna
3159,SALAS CUADROS JORGE AUGUSTO,239996,ACCION POPULAR,0,0,0,0,0,0,0,0.0,0.0,True
2892,GUEVARA VERGARA CLODOMIRO,41690329,RENACIMIENTO UNIDO NACIONAL,0,0,0,0,0,0,0,0.0,0.0,True
3161,HUACRE MEZA BRENDA JEANETTE,45074945,RENACIMIENTO UNIDO NACIONAL,0,0,0,0,0,0,0,0.0,0.0,True


In [ ]:
cong['ingreso mensual'] = cong['total_ingreso'] / 12
cong['ingreso_menos_p'] = (cong['total_ingreso'] - cong['ingreso_publico'])/ 12

Otra forma es utilizando el ```eval```: 

In [61]:
cong['new_ingreso_mp'] = cong.eval('(total_ingreso-ingreso_publico)/12')
cong['new_ingreso_mensual'] = cong.eval('total_ingreso/12')


Podemos crear variables categóricas en base a variables continuas con ```cut```: 

In [ ]:
categ_labels = ['cat_1', 'cat_2', 'cat_3', 'cat_4']
categ_bins = [-1, 10000, 50000, 100000, 200000000]

In [ ]:
cong2['cat_ingreso'] = pd.cut(cong2['total_ingreso'],
                              bins = categ_bins, labels = categ_labels)

Tarea: Crear cuantiles (qcut)

### Agregar datos

Muchas veces vamos a querer una agregación de los datos que nos resuma algunas de las variables en sus medidas de tendencia central, o las que nos interese. 

La forma más general de agregar los datos es la siguiente: 
    

In [9]:
cong.median()

dni                     23921498.0
total_ingreso              26859.0
ingreso_publico                0.0
ingreso_privado                0.0
renta_publico                  0.0
renta_privado                  0.0
otro_ingreso_publico           0.0
otro_ingreso_privado           0.0
dtype: float64

In [92]:

cong['total_ingreso'].mean()

88349.71441495778

In [94]:
cong[cong['partido'] == "PARTIDO MORADO"]['total_ingreso'].mean()

69270.45505617978

En general, podemos hacer todas estas agregaciones: 

* `mean` - promedio
* `median` - mediana
* `sum` - suma
* `min` - mínimo
* `max` - máximo
* `quantile` - nos da el cuantil que le pedimos
* `std` - desviación standar
* `var` - varianza
* `count` - produce el número de valores no missing
* `describe` - nos da varias de las medidas de arriba


### OJO: Podemos hacer agregaciones de ó columnas ó filas. 

Para hacer agregaciones por columnas: ```axis = 0``` ó ```axis = 'index' ```
Para hacer agregaciones por filas: ```axis = 1``` ó ```axis = 'columns' ```

In [104]:
## Sumando todos los ingresos por congresista
cong['otro_total'] = cong[['ingreso_publico',
       'ingreso_privado', 'renta_publico', 'renta_privado',
       'otro_ingreso_publico', 'otro_ingreso_privado']].sum(axis = 1)

In [110]:
cong[['ingreso_publico',
       'ingreso_privado', 'renta_publico', 'renta_privado',
       'otro_ingreso_publico', 'otro_ingreso_privado']].quantile(q = 0.8, axis = 0)

ingreso_publico         24000.0
ingreso_privado         11100.0
renta_publico               0.0
renta_privado           14400.0
otro_ingreso_publico        0.0
otro_ingreso_privado        0.0
Name: 0.8, dtype: float64

In [134]:
cong[['total_ingreso', 'partido']].groupby('partido').mean()

,total_ingreso
partido,
ACCION POPULAR,59437.463277
ALIANZA PARA EL PROGRESO,124155.849162
AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL,80444.471338
DEMOCRACIA DIRECTA,25121.864407
"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",28326.472393
FRENTE POPULAR AGRICOLA FIA DEL PERU - FREPAP,19786.235955
FUERZA POPULAR,94359.858757
JUNTOS POR EL PERU,48673.322034
PARTIDO APRISTA PERUANO,122434.052632


In [135]:
(cong[['total_ingreso', 'partido']].groupby('partido').max().reset_index())

#.sort_values('total_ingreso')

,partido,total_ingreso
0,ACCION POPULAR,605574
1,ALIANZA PARA EL PROGRESO,9571448
2,AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL,1592210
3,DEMOCRACIA DIRECTA,460922
4,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",278606
5,FRENTE POPULAR AGRICOLA FIA DEL PERU - FREPAP,102000
6,FUERZA POPULAR,2750000
7,JUNTOS POR EL PERU,336393
8,PARTIDO APRISTA PERUANO,970021
9,PARTIDO DEMOCRATICO SOMOS PERU,567989


Si bien hay varias formas, la más general es la siguiente:

In [136]:
nueva_agregacion = cong.groupby(['partido']).agg(
{'total_ingreso': ['mean', 'max','min'], 
 'ingreso_privado': ['mean', 'median']}
)

In [137]:
nueva_agregacion

total_ingreso                  \
                                                         mean       max   min   
partido                                                                         
ACCION POPULAR                                   59437.463277    605574     0   
ALIANZA PARA EL PROGRESO                        124155.849162   9571448     0   
AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL      80444.471338   1592210     0   
DEMOCRACIA DIRECTA                               25121.864407    460922     0   
EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD   28326.472393    278606     0   
FRENTE POPULAR AGRICOLA FIA DEL PERU - FREPAP    19786.235955    102000     0   
FUERZA POPULAR                                   94359.858757   2750000     0   
JUNTOS POR EL PERU                               48673.322034    336393     0   
PARTIDO APRISTA PERUANO                         122434.052632    970021  4200   
PARTIDO DEMOCRATICO SOMOS PERU                   57238.775281    567989     0   
PARTIDO FRENTE DE LA ESPERANZA 2021              18141.125000    265901     0   
PARTIDO MORADO                                   69270.455056   1246086     0   
PARTIDO NACIONALISTA PERUANO                     42424.757062    487064     0   
PARTIDO POLITICO NACIONAL PERU LIBRE             43010.303371    852600     0   
PARTIDO POLÍTICO CONTIGO                         64756.382353    879394     0   
PARTIDO POPULAR CRISTIANO - PPC                  48740.570621   1798250     0   
PERU PATRIA SEGURA                               69610.990385   2241500     0   
PODEMOS PERU                                    161985.821839  12428083     0   
RENACIMIENTO UNIDO NACIONAL                      86746.683616   7740000     0   
RENOVACION POPULAR                              156966.491525  10998927     0   
UNION POR EL PERU                                49173.114650    542671     0   
VICTORIA NACIONAL                               446236.639752  57560800     0   

                                               ingreso_privado         
                                                          mean median  
partido                                                                
ACCION POPULAR                                    12390.203390      0  
ALIANZA PARA EL PROGRESO                          32321.284916      0  
AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL       15340.000000      0  
DEMOCRACIA DIRECTA                                 7925.909605      0  
EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD     4514.509202      0  
FRENTE POPULAR AGRICOLA FIA DEL PERU - FREPAP      1044.376404      0  
FUERZA POPULAR                                    16421.644068      0  
JUNTOS POR EL PERU                                 9023.209040      0  
PARTIDO APRISTA PERUANO                           93405.052632  17638  
PARTIDO DEMOCRATICO SOMOS PERU                     7097.713483      0  
PARTIDO FRENTE DE LA ESPERANZA 2021                2340.500000      0  
PARTIDO MORADO                                    17235.814607      0  
PARTIDO NACIONALISTA PERUANO                       4284.118644      0  
PARTIDO POLITICO NACIONAL PERU LIBRE               8802.792135      0  
PARTIDO POLÍTICO CONTIGO                          15342.764706      0  
PARTIDO POPULAR CRISTIANO - PPC                   20581.135593      0  
PERU PATRIA SEGURA                                15355.000000      0  
PODEMOS PERU                                      46638.022989      0  
RENACIMIENTO UNIDO NACIONAL                       11201.305085      0  
RENOVACION POPULAR                                17748.508475      0  
UNION POR EL PERU                                  7968.165605      0  
VICTORIA NACIONAL                                105948.465839      0

In [138]:
## El reset index es importante!
nueva_agregacion = cong.groupby(['partido']).agg(
{'total_ingreso': ['mean', 'max'], 
 'ingreso_privado': ['mean', 'median']}
).reset_index()



In [139]:
nueva_agregacion

partido  total_ingreso            \
                                                             mean       max   
0                                   ACCION POPULAR   59437.463277    605574   
1                         ALIANZA PARA EL PROGRESO  124155.849162   9571448   
2      AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL   80444.471338   1592210   
3                               DEMOCRACIA DIRECTA   25121.864407    460922   
4   EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD   28326.472393    278606   
5    FRENTE POPULAR AGRICOLA FIA DEL PERU - FREPAP   19786.235955    102000   
6                                   FUERZA POPULAR   94359.858757   2750000   
7                               JUNTOS POR EL PERU   48673.322034    336393   
8                          PARTIDO APRISTA PERUANO  122434.052632    970021   
9                   PARTIDO DEMOCRATICO SOMOS PERU   57238.775281    567989   
10             PARTIDO FRENTE DE LA ESPERANZA 2021   18141.125000    265901   
11                                  PARTIDO MORADO   69270.455056   1246086   
12                    PARTIDO NACIONALISTA PERUANO   42424.757062    487064   
13            PARTIDO POLITICO NACIONAL PERU LIBRE   43010.303371    852600   
14                        PARTIDO POLÍTICO CONTIGO   64756.382353    879394   
15                 PARTIDO POPULAR CRISTIANO - PPC   48740.570621   1798250   
16                              PERU PATRIA SEGURA   69610.990385   2241500   
17                                    PODEMOS PERU  161985.821839  12428083   
18                     RENACIMIENTO UNIDO NACIONAL   86746.683616   7740000   
19                              RENOVACION POPULAR  156966.491525  10998927   
20                               UNION POR EL PERU   49173.114650    542671   
21                               VICTORIA NACIONAL  446236.639752  57560800   

   ingreso_privado         
              mean median  
0     12390.203390      0  
1     32321.284916      0  
2     15340.000000      0  
3      7925.909605      0  
4      4514.509202      0  
5      1044.376404      0  
6     16421.644068      0  
7      9023.209040      0  
8     93405.052632  17638  
9      7097.713483      0  
10     2340.500000      0  
11    17235.814607      0  
12     4284.118644      0  
13     8802.792135      0  
14    15342.764706      0  
15    20581.135593      0  
16    15355.000000      0  
17    46638.022989      0  
18    11201.305085      0  
19    17748.508475      0  
20     7968.165605      0  
21   105948.465839      0

In [115]:
nueva_agregacion.columns

MultiIndex([(        'partido',       ''),
            (  'total_ingreso',   'mean'),
            (  'total_ingreso',    'max'),
            ('ingreso_privado',   'mean'),
            ('ingreso_privado', 'median')],
           )

In [130]:
nueva_agregacion[(  'total_ingreso',   'mean')]

0      59437.463277
1     124155.849162
2      80444.471338
3      25121.864407
4      28326.472393
5      19786.235955
6      94359.858757
7      48673.322034
8     122434.052632
9      57238.775281
10     18141.125000
11     69270.455056
12     42424.757062
13     43010.303371
14     64756.382353
15     48740.570621
16     69610.990385
17    161985.821839
18     86746.683616
19    156966.491525
20     49173.114650
21    446236.639752
Name: (total_ingreso, mean), dtype: float64

In [ ]:
nueva_agregacion.columns = [''.join(col).strip() for col in nueva_agregacion.columns.values]

In [140]:
new_col = []
for col in nueva_agregacion.columns.values:
    a = ''.join(col).strip()
    new_col.append(a)
nueva_agregacion.columns = new_col

In [141]:
nueva_agregacion


,partido,total_ingresomean,total_ingresomax,ingreso_privadomean,ingreso_privadomedian
0,ACCION POPULAR,59437.463277,605574,12390.203390,0
1,ALIANZA PARA EL PROGRESO,124155.849162,9571448,32321.284916,0
2,AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL,80444.471338,1592210,15340.000000,0
3,DEMOCRACIA DIRECTA,25121.864407,460922,7925.909605,0
4,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",28326.472393,278606,4514.509202,0
5,FRENTE POPULAR AGRICOLA FIA DEL PERU - FREPAP,19786.235955,102000,1044.376404,0
6,FUERZA POPULAR,94359.858757,2750000,16421.644068,0
7,JUNTOS POR EL PERU,48673.322034,336393,9023.209040,0
8,PARTIDO APRISTA PERUANO,122434.052632,970021,93405.052632,17638
9,PARTIDO DEMOCRATICO SOMOS PERU,57238.775281,567989,7097.713483,0


Una nueva forma de ordenar nuestro código es ponerlo entre paréntesis y separarlo por "puntitos"

In [84]:
nueva_agregacion = (cong
                    .groupby(['partido'])
                    .agg(
                    {'total_ingreso': ['mean', 'max'], 
                     'ingreso_privado': ['mean', 'median']}
                   ).reset_index())



### Juntar datos (merge)

A veces tenemos los datos que nos interesan, esparcidos en diferentes dataframes.   
1er paso para hacer la unión de datos: Identificar el identificador único!



In [131]:
cong_1 = cong[['dni', 'total_ingreso']]

cong_2 = cong[['dni', 'partido']]


La sintaxis más general de un merge es la siguiente: 

``` python
pd.merge(left = left_df,
         right=right_df,
         left_on=colname, 
         right_on=colname,
         how=merge_type)

```

El merge_type (o tipo de merge puede ser):

- left: quedan todas las observaciones (por keys) de las mano izquierda
- right: quedan todas las observaciones (por keys) de la mano derecha
- outer: es la unión de los keys de ambas bases
- inner: es la intersección de los keys de ambas bases. 


In [143]:
resultado_merge = pd.merge(cong_1,
                     cong_2,
                    left_on = 'dni',
                    right_on = 'dni',
                    how = 'inner', indicator = True)
resultado_merge

,dni,total_ingreso,partido,_merge
0,239996,0,ACCION POPULAR,both
1,41690329,0,RENACIMIENTO UNIDO NACIONAL,both
2,45074945,0,RENACIMIENTO UNIDO NACIONAL,both
3,41738609,0,RENACIMIENTO UNIDO NACIONAL,both
4,42818690,0,RENACIMIENTO UNIDO NACIONAL,both
...,...,...,...,...
3311,19873483,7740000,RENACIMIENTO UNIDO NACIONAL,both
3312,17903382,9571448,ALIANZA PARA EL PROGRESO,both
3313,7818712,10998927,RENOVACION POPULAR,both
3314,7246887,12428083,PODEMOS PERU,both


In [86]:
solo_fp = cong_2.loc[cong2['partido'] == 'FUERZA POPULAR', :]

In [88]:
resultado_merge_fp = pd.merge(cong_1,
                     solo_fp,
                    left_on = 'dni',
                    right_on = 'dni',
                    how = 'inner')
resultado_merge_fp

,dni,total_ingreso,partido
0,508595,0,FUERZA POPULAR
1,41221397,0,FUERZA POPULAR
2,46994878,0,FUERZA POPULAR
3,42344527,0,FUERZA POPULAR
4,7721458,0,FUERZA POPULAR
...,...,...,...
172,8232920,444400,FUERZA POPULAR
173,508734,500000,FUERZA POPULAR
174,8803658,515605,FUERZA POPULAR
175,17896798,884784,FUERZA POPULAR
